In [ ]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

In [ ]:
# Create a test collection and stage a test item
collection = create_test_collection()
item = stage_test_item()

In [ ]:
import requests

# In this cell, we will call all the catalog GET endpoints and check that 
# the stac authentication extension is set in each JSON output value.

# We extract the endpoint paths from the openapi.json specification
openapi_endpoint = "/api" # if local_mode else "/docs/openapi.json" # TEMP !!
openapi = requests.get(f"{stac_client.href_catalog}{openapi_endpoint}", **apikey_headers).json()
catalog_endpoints = [
    key
    for key, value in openapi["paths"].items()
    if key.startswith("/catalog") and ("get" in value)
]

# Call the endpoints with and without the owner_id: prefix
if cluster_mode:
    implicit_ownerid = []
    for endpoint in catalog_endpoints:
         implicit_ownerid.append(endpoint)
         if "{owner_id}:{collection_id}" in endpoint:
             implicit_ownerid.append(endpoint.replace("{owner_id}:{collection_id}", "{collection_id}"))
    catalog_endpoints = implicit_ownerid

# Now call each endpoint
for endpoint in catalog_endpoints:

    # Don't call these ones, they don't implement the authentication extension
    if endpoint in [
        "/catalog/api", # openapi.json specification
        "/catalog/api.html", # swagger page
        "/catalog/docs/oauth2-redirect", # don't really know what this is
        "/catalog/conformance", # returns the conformsTo links
        "/catalog/search", # TO be discussed
        # don't really know what the queryables do, but the authentication extension 
        # is not implemented for these.
        "/catalog/queryables",
        "/catalog/collections/{owner_id}:{collection_id}/queryables",
        "/catalog/collections/{collection_id}/queryables",
    ]:
        print(f"{endpoint}\n(ignored)\n")
        continue

    # Replace the {owner_id}, {collection_id}, {item_id} by the right values
    endpoint = endpoint.format(
        owner_id=stac_client.owner_id, collection_id=collection.id, item_id=item.id,
    )
    print(endpoint)

    # Call the GET endpoint
    stac = requests.get(f"{stac_client.href_catalog}{endpoint}", **apikey_headers).json()

    # Print the authentication extension property values
    for field in "stac_extensions", "auth:schemes", "links", "assets":
        value = stac.get(field, None)
        if value:
            print(f"{field}: {json.dumps(value, indent=2)}")
    print()

    # In cluster mode, we check that the extension is implemented
    if cluster_mode:
        assert "https://stac-extensions.github.io/authentication/v1.1.0/schema.json" in stac["stac_extensions"]
        assert "auth:schemes" in stac
        for link in stac.get("links", []):
            assert link["auth:refs"] == ["apikey"]
        for asset in stac.get("assets", {}).values():
            assert asset["auth:refs"] == ["apikey"]

    # In local mode, we check that it is NOT implemented
    else:
        assert "https://stac-extensions.github.io/authentication/v1.1.0/schema.json" not in stac.get("stac_extensions", [])
        assert "auth:schemes" not in stac

        for link in stac.get("links", []):
            assert "auth:refs" not in link

        for asset in stac.get("assets", {}).values():
            assert "auth:refs" not in asset